[![logo](https://climate.copernicus.eu/sites/default/files/custom-uploads/branding/LogoLine_horizon_CAMS.png)](https://atmosphere.copernicus.eu)

# Policy tools

 **Run the tutorial via free cloud platforms**: [![binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/ecmwf-training/cams-act6/main?labpath=05-cams-mos/policy_tools.ipynb)
[![kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ecmwf-training/cams-act6/blob/main/05-cams-mos/policy_tools.ipynb)
[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ecmwf-training/cams-act6/blob/main/05-cams-mos/policy_tools.ipynb)

The policy tools provide two APIs to consume the policy data using a client software, such as this notebook, instead of by a person.

## APIs

### Air Control Toolbox (ACT)
https://policy.atmosphere.copernicus.eu/data/{yyyymm}/CHIMERE/{model}_{city}_{yyyymmdd}.json

### Source Allocation
https://policy.atmosphere.copernicus.eu/data/climato/{city}-{parameter}-{model}-{year}.json

## Questions
1. Which are the 10 cities where the dust share in PM10 is highest ?
2. On average between 2021 and 2023, which country contributed most to PM10 pollution in Prague ?
3. Which are the 10 cities where the maritime sector had the largest impact on ozone daily max between June and August 2024 ?

### Import libraries

In [15]:
import os
import sys
import json
import requests
import urllib.request
import pandas as pd
import math
import datetime
from math import ceil, sqrt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

### Question 1
Here we want to answer the question: which are the 10 cities where the dust share in PM10 is highest ? We could answer this question by using the [Policy Tools](https://policy.atmosphere.copernicus.eu/daily_source_attribution/sector_apportionment.php?date=2024-09-10) website and selecting "Sector apportionment" in the "Daily Source Attribution" menu. In the web page we can hereafter select the city and the pollutant. We can use the API to send several requests and process the data to answer more complex questions such as the one we are dealing with here. 

In [23]:
with open('cities.txt', 'r', encoding="utf8") as file:
    lines = [line.strip() for line in file]

In [26]:
num_cities = len(lines)
num_cities

80

In [67]:
def act_url(city, day):
    return 'https://policy.atmosphere.copernicus.eu/data/' + day[:6] + '/CHIMERE/ACT_' + city + '_' + day + '.json'

In [117]:
day = '20240910'
act_forecast_url = act_url('Rome', day)
act_forecast_url

'https://policy.atmosphere.copernicus.eu/data/202409/CHIMERE/ACT_Rome_20240910.json'

In [69]:
act_file_name = act_forecast_url.split('/')[-1:].pop()
act_file_name

'ACT_Rome_20240910.json'

In [80]:
with urllib.request.urlopen(act_forecast_url) as url:
    act_data = json.load(url)
    #print(act_data)

We can store the json file

In [81]:
with open(act_file_name, 'w') as f:
    json.dump(act_data, f)

We parse the json file to extract the information we need to answer our question

In [118]:
for item in act_data.get(day):
    name = item['name'] 
    polluttant = item['poll']
    if (polluttant == 'PM10'):
        hOut = sum(item['hOut'])
        print('Sector: {:s}, PM10: {:.2f}'.format(name, hOut))

Sector: Shipping, PM10: 22.69
Sector: Natural, PM10: 385.53
Sector: Others, PM10: 26.84
Sector: Agriculture, PM10: 37.39
Sector: Industry, PM10: 22.96
Sector: Residential, PM10: 126.53
Sector: Residual, PM10: -10.24
Sector: Traffic, PM10: 20.61


### Question 2
Here the question is: on average between 2021 and 2023, which country contributed most to PM10 pollution in Prague ? We could answer this question using the [policy tools](https://policy.atmosphere.copernicus.eu/daily_source_attribution/country_contribution.php?date=2024-09-10) by selecting "Country contribution" in the "Yearly Air Pollution Analysis" menu. Afterward we can select the city and the pollutant. As for question 1, we can use the API to send several requests, e.g. for several cities, and process the data to answer more complex questions than the one we are dealing with here. (there is no json file in the web page)

In [61]:
def sa_url(city, year, polluttant):
    return 'https://policy.atmosphere.copernicus.eu/data/climato/' + city + '-' + polluttant + '-ACT-' + year + '.json'

In [62]:
sa_data_url = sa_url('Oslo', '2024', 'PM10')
sa_data_url

'https://policy.atmosphere.copernicus.eu/data/climato/Oslo-PM10-ACT-2024.json'

In [64]:
sa_file_name = sa_data_url.split('/')[-1:].pop()
sa_file_name

'Oslo-PM10-ACT-2024.json'

In [65]:
 with urllib.request.urlopen(sa_data_url) as url:
    sa_data = json.load(url)
    #print(data)

In [66]:
with open(sa_file_name, 'w') as f:
    json.dump(sa_data, f)

### Question 3
Here the question is: which are the 10 cities where the maritime sector had the largest impact on ozone daily max between June and August 2024 ? The maritime sector is labeled as Ship or SHP. In order to answer this question we use again the 

## References
* [Colette et al. - Air Control Toolbox (ACT_v1.0): a flexible surrogate model to explore mitigation scenarios in air quality forecasts](https://gmd.copernicus.org/articles/15/1441/2022/)